In [8]:
import pandas as pd
import numpy as np
import re
pd.options.display.max_rows = 4000

In [9]:
######### Read Data set 1
Energy = pd.read_excel('Energy Indicators.xls', skipfooter=38, header=17)

# Rename columns
Energy.rename(columns={Energy.columns[2]:'Country'}, inplace=True)
Energy.rename(columns={Energy.columns[3]:'Energy Supply'}, inplace=True)
Energy.rename(columns={Energy.columns[4]:'Energy Supply per Capita'}, inplace=True)
Energy.rename(columns={Energy.columns[5]:'% Renewable'}, inplace=True)

# Restrict columns
use_columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
Energy = Energy[use_columns]

# Convert units
Energy['Energy Supply'] = 1000000* Energy['Energy Supply']

# Replace ..... values with np.nan
Energy.replace('\.+', np.nan, regex=True, inplace=True)

# Rename countires
cname_change_list = {"Republic of Korea": "South Korea", "United States of America": "United States",\
                     "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",\
                     "China, Hong Kong Special Administrative Region": "Hong Kong" \
                    }

for i,cname in enumerate( Energy['Country']):
    for cname_key, cname_new in cname_change_list.items():
        if re.match(cname_key, cname):
            Energy['Country'][i] = cname_new

# Change index to country names
Energy = Energy.set_index( ['Country'] )

# Split by (  and remove numbers by blanks. Note the order.
names_ids = Energy.index.str.replace('\d+', '')
names_ids = names_ids.str.split( '\s\(' )
Energy.index = names_ids.str[0]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
####### Read Data set 2
GDP = pd.read_csv('world_bank.csv', skiprows=4)
# Rename Label
GDP.rename(columns={GDP.columns[0]:'Country'}, inplace=True)
# Rename countires
cname_change_list = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
                    
for i,cname in enumerate( GDP['Country']):
    for cname_key, cname_new in cname_change_list.items():
        if re.match(cname_key, cname):
            GDP['Country'][i] = cname_new
GDP = GDP.set_index( ['Country'])

# Remove unnecessary columns
columns_to_keep = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
GDP = GDP[columns_to_keep]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [11]:
###### Read Data set 3
ScimEn = pd.read_excel('scimagojr-3.xlsx', skipfooter=0, header=0)
# Set index
ScimEn = ScimEn.set_index( ['Country'] )

In [12]:
######## Merge data sets
df2 = pd.merge(pd.merge(ScimEn,Energy,left_on='Country',right_index=True,how ='outer'),GDP,left_on='Country',right_on='Country',how ='outer')
df2 = df2.sort_values( by=['Rank'], ascending=True)

In [13]:
df2.head()
# df2.info()

,Country,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,China,1.0,127050.0,126767.0,597237.0,411683.0,4.70,138.0,1.271910e+11,93.0,...,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
1,United States,2.0,96661.0,94747.0,792274.0,265436.0,8.20,230.0,9.083800e+10,286.0,...,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
2,Japan,3.0,30504.0,30287.0,223024.0,61554.0,7.31,134.0,1.898400e+10,149.0,...,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
3,United Kingdom,4.0,20944.0,20357.0,206091.0,37874.0,9.84,139.0,7.920000e+09,124.0,...,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
4,Russian Federation,5.0,18534.0,18301.0,34266.0,12422.0,1.85,57.0,3.070900e+10,214.0,...,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
